### Load Test Data and Trained APG Sampler

In [1]:
%matplotlib inline
import os
import torch
import numpy as np
from apgs.bmnist.apg_training import init_models
from apgs.bmnist.affine_transformer import Affine_Transformer

CUDA = torch.cuda.is_available()
device = torch.device('cuda:1')
data_dir = '../../data/bmnist/'
timesteps, num_digits, frame_pixels, mnist_pixels, num_hidden_digit, num_hidden_coor, z_where_dim, z_what_dim = 10, 3, 96, 28, 400, 400, 2, 10
data_paths = []
for file in os.listdir(data_dir + 'train/'):
    data_paths.append(os.path.join(data_dir, 'train', file))
model_version = 'apg-bmnist-num_sweeps=6-num_samples=16'
models = init_models(frame_pixels, mnist_pixels, num_hidden_digit, num_hidden_coor, z_where_dim, z_what_dim, CUDA, device, model_version, lr=None)
AT = Affine_Transformer(frame_pixels, mnist_pixels, CUDA, device)  

In [5]:
models[0].

<bound method Module.named_modules of Enc_coor(
  (enc_hidden): Sequential(
    (0): Linear(in_features=4761, out_features=400, bias=True)
    (1): ReLU()
  )
  (where_mean): Sequential(
    (0): Linear(in_features=400, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=2, bias=True)
    (3): Tanh()
  )
  (where_log_std): Sequential(
    (0): Linear(in_features=400, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=2, bias=True)
  )
)>

In [2]:
for name, param in models[0].named_parameters():
    if param.requires_grad:
        print(name, param)

enc_hidden.0.weight Parameter containing:
tensor([[ 0.0334,  0.0424,  0.0632,  ...,  0.0444,  0.0370,  0.0373],
        [-0.2199, -0.2690, -0.3178,  ...,  0.0028, -0.1610, -0.2897],
        [ 0.2490,  0.1562,  0.1423,  ..., -0.0612, -0.0576, -0.0477],
        ...,
        [ 0.0062, -0.0218, -0.0182,  ...,  0.0676,  0.0982,  0.0584],
        [-0.0490, -0.0709, -0.0890,  ...,  0.1896,  0.2327,  0.2719],
        [ 0.0649,  0.0938,  0.0653,  ..., -0.0547, -0.0361, -0.0766]],
       device='cuda:1', requires_grad=True)
enc_hidden.0.bias Parameter containing:
tensor([-0.0347,  0.0077,  0.0497, -0.0162,  0.0438,  0.0529,  0.0031,  0.0388,
         0.0461,  0.0406, -0.0161,  0.0421,  0.0421,  0.0428,  0.0392,  0.0195,
         0.0441, -0.0119, -0.0254, -0.0185, -0.0178,  0.0373,  0.0472,  0.0522,
         0.0391,  0.0519,  0.0464,  0.0372,  0.0484,  0.0386,  0.0594,  0.0084,
         0.0448,  0.0484, -0.0053,  0.0322, -0.0590,  0.0314,  0.0374,  0.0093,
         0.0013,  0.0435,  0.0460, -0.03

### Visualize Samples

In [ ]:
from apgs.resampler import Resampler
from apgs.bmnist.objectives import apg_objective
from apgs.bmnist.evaluation import viz_samples
from random import shuffle
batch_size, num_sweeps = 5, 10
shuffle(data_paths)
data = torch.from_numpy(np.load(data_paths[0])).float()[:batch_size].unsqueeze(0)
mnist_mean = torch.from_numpy(np.load('mnist_mean.npy')).float()
mnist_mean = mnist_mean.repeat(batch_size, num_digits, 1, 1).unsqueeze(0)
if CUDA:
    data = data.cuda().to(device)
    mnist_mean = mnist_mean.cuda().to(device)
result_flags = {'loss_required' : False, 'ess_required' : False, 'mode_required' : True, 'density_required': False}
trace = apg_objective(models, AT, data, num_digits, result_flags, num_sweeps, Resampler('systematic', 1, CUDA, device), mnist_mean)
viz_samples(data.squeeze(0).cpu(), trace, num_sweeps, num_digits)

### Computing log joint across all methods

In [ ]:
from apgs.bmnist.evaluation import density_all_instances
from random import shuffle
sample_size, num_sweeps = 1000, 10
lf_step_size, lf_num_steps, bpg_factor = 1e-4, [1, 5, 10], 100
density_all_instances(models, AT, data_paths, sample_size, num_digits, z_where_dim, z_what_dim, num_sweeps, lf_step_size, lf_num_steps, bpg_factor, CUDA, device)